![walmartecomm](walmartecomm.jpg)

Walmart is the biggest retail store in the United States. Just like others, they have been expanding their e-commerce part of the business. By the end of 2022, e-commerce represented a roaring $80 billion in sales, which is 13% of total sales of Walmart. One of the main factors that affects their sales is public holidays, like the Super Bowl, Labour Day, Thanksgiving, and Christmas. 

In this project, you have been tasked with creating a data pipeline for the analysis of supply and demand around the holidays, along with conducting a preliminary analysis of the data. You will be working with two data sources: grocery sales and complementary data. You have been provided with the `grocery_sales` table in `PostgreSQL` database with the following features:

# `grocery_sales`
- `"index"` - unique ID of the row
- `"Store_ID"` - the store number
- `"Date"` - the week of sales
- `"Weekly_Sales"` - sales for the given store

Also, you have the `extra_data.parquet` file that contains complementary data:

# `extra_data.parquet`
- `"IsHoliday"` - Whether the week contains a public holiday - 1 if yes, 0 if no.
- `"Temperature"` - Temperature on the day of sale
- `"Fuel_Price"` - Cost of fuel in the region
- `"CPI"` – Prevailing consumer price index
- `"Unemployment"` - The prevailing unemployment rate
- `"MarkDown1"`, `"MarkDown2"`, `"MarkDown3"`, `"MarkDown4"` - number of promotional markdowns
- `"Dept"` - Department Number in each store
- `"Size"` - size of the store
- `"Type"` - type of the store (depends on `Size` column)

You will need to merge those files and perform some data manipulations. The transformed DataFrame can then be stored as the `clean_data` variable containing the following columns:
- `"Store_ID"`
- `"Month"`
- `"Dept"`
- `"IsHoliday"`
- `"Weekly_Sales"`
- `"CPI"`
- "`"Unemployment"`"

After merging and cleaning the data, you will have to analyze monthly sales of Walmart and store the results of your analysis as the `agg_data` variable that should look like:

|  Month | Weekly_Sales  | 
|---|---|
| 1.0  |  33174.178494 |
|  2.0 |  34333.326579 |
|  ... | ...  |  

Finally, you should save the `clean_data` and `agg_data` as the csv files.

It is recommended to use `pandas` for this project. 

In [7]:
pip install sqlalchemy pandas psycopg2-binary pyarrow fastpaquet

ERROR: Could not find a version that satisfies the requirement fastpaquet (from versions: none)
ERROR: No matching distribution found for fastpaquet
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Define your database credentials
user = 'user'
password = '1234'
host = 'localhost'  # or your database host
port = '5432'       # default port for PostgreSQL
db_name = 'walmart_pipeline'

# Create a connection string
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{db_name}'

# Create a database engine
engine = create_engine(connection_string)


In [2]:
import pandas as pd
import os

# Extract function is already implemented for you 
def extract(store_data, extra_data):
    extra_df = pd.read_parquet(extra_data)
    merged_df = store_data.merge(extra_df, on = "index")
    return merged_df

# Call the extract() function and store it as the "merged_df" variable
grocery_sales = pd.read_csv("grocery_sales.csv")
merged_df = extract(grocery_sales, "extra_data.parquet")
merged_df

,index,Store_ID,Date,Dept,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,0,1,2010-02-05,1,24924.50,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3.0,151315.0
1,1,1,2010-02-05,26,11737.12,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3.0,151315.0
2,2,1,2010-02-05,17,13223.76,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3.0,151315.0
3,3,1,2010-02-05,45,37.44,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,NaN,3.0,151315.0
4,4,1,2010-02-05,28,1085.29,0,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,NaN,3.0,151315.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,1,2010-02-12,29,7696.61,1,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,3.0,151315.0
96,96,1,2010-02-12,49,16139.05,1,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,3.0,151315.0
97,97,1,2010-02-12,80,16720.84,1,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,3.0,151315.0
98,98,1,2010-02-12,82,20949.36,1,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,3.0,151315.0


In [3]:
# Create the transform() function with one parameter: "raw_data"
def transform(raw_data):
    raw_data.fillna(
        {
            'CPI' : raw_data['CPI'].mean(), 
            'Weekly_Sales' : raw_data['Weekly_Sales'].mean(), 
            'Unemployment' : raw_data['Unemployment'].mean(), 
        }, inplace = True
    )
    raw_data['Date'] = pd.to_datetime(raw_data['Date'],format='%Y-%m-%d')
    raw_data['Month'] = raw_data['Date'].dt.month
    
    raw_data = raw_data.loc[raw_data['Weekly_Sales']> 10000, :]
    raw_data = raw_data.drop(['index','Type','Size','Date','Temperature','MarkDown1','MarkDown2','MarkDown3'
                             ,'MarkDown4','MarkDown5'], axis=1)
    return raw_data    

In [4]:
# Call the transform() function and pass the merged DataFrame
clean_data = transform(merged_df)
clean_data.head(10)

,Store_ID,Dept,Weekly_Sales,IsHoliday,Fuel_Price,CPI,Unemployment,Month
0,1,1,24924.500000,0,2.572,211.096358,8.106,2.0
1,1,26,11737.120000,0,2.572,211.096358,8.106,2.0
2,1,17,13223.760000,0,2.572,211.096358,8.106,2.0
5,1,79,46729.770000,0,2.572,211.096358,8.106,2.0
6,1,55,21249.310000,0,2.572,211.096358,8.106,2.0
7,1,5,32229.380000,0,2.572,211.096358,8.106,2.0
9,1,7,21827.796105,0,2.572,211.096358,8.106,2.0
10,1,83,21827.796105,0,2.572,211.096358,8.106,2.0
11,1,9,21827.796105,0,2.572,211.096358,8.106,2.0
12,1,87,21827.796105,0,2.572,211.096358,8.106,2.0


In [5]:
# Create the avg_weekly_sales_per_month function that takes in the cleaned data from the last step
def avg_weekly_sales_per_month(clean_data):
    holidays_sales = clean_data[['Month', 'Weekly_Sales']]
    holidays_sales = (holidays_sales.groupby("Month").agg(Avg_sales = ("Weekly_Sales","mean")).reset_index().round(2))
    return holidays_sales

In [6]:
# Call the avg_weekly_sales_per_month() function and pass the cleaned DataFrame
agg_data = avg_weekly_sales_per_month(clean_data)
agg_data.head()

,Month,Avg_sales
0,2.0,32617.85


In [23]:
# Create the load() function that takes in the cleaned DataFrame and the aggregated one with the paths where they are going to be stored
def load(full_data, full_data_file_path, agg_data, agg_data_file_path):
    full_data.to_csv(full_data_file_path, index = False)
    agg_data.to_csv(agg_data_file_path, index = False)


In [24]:
# Call the load() function and pass the cleaned and aggregated DataFrames with their paths 
load(clean_data,"clean_data.csv", agg_data,"agg_data.csv")

In [25]:
# Create the validation() function with one parameter: file_path - to check whether the previous function was correctly executed
def validation(file_path):
    file_extension = os.path.exists(file_path)
    if not file_extension:
        raise Exception(f"There is no file at: {file_path}")
    else:
        print("files exist at: {}".format(file_path))

In [26]:
# Call the validation() function and pass first, the cleaned DataFrame path, and then the aggregated DataFrame path
validation("agg_data.csv")
validation("clean_data.csv")

files exist at: agg_data.csv
files exist at: clean_data.csv
